In [ ]:
using JLD2
using DataFrames
using PlotlyJS
using StatsBase
using Dates
include("C://repos//trade//connector//ts2hdf5//client.jl")
include("C://repos//trade//common//utils.jl")
;

In [ ]:
const Client = ClientTsHdf5;

In [ ]:
df_price = Client.query(Dict(
    "exchange"=> "bitfinex",
    "measurement_name"=> "trade bars",
    "asset"=> "ethusd",
    "unit" =>"ethusd",
    "information" => "price",
    "col" => "price"
))
df_price = DataFrame(df_price, ["ts", "price"]);

In [ ]:
df = Client.query(Dict(
        "measurement_name"=> "order book",
    "delta_size_ratio"=> 0.5,
    "exchange"=> "bitfinex",
    "unit"=> "size_ewm_sum",
    "information"=> "bid_buy_size_imbalance_ratio",
    "asset"=> "ethusd"
))
df = DataFrame(df, ["ts", "bid_buy_size_imbalance_ratio"]);

In [ ]:
dfj = outerjoin(df, df_price, on=:ts )
start=maximum([df[1, "ts"], df_price[1, "ts"]])
end_=minimum([df[end, "ts"], df_price[end, "ts"]])
dfj = dfj[(dfj[:, "ts"] .>= start) .& (dfj[:, "ts"] .< end_), :]
sort!(dfj, ["ts"])
dfj[:, "price"] .= ffill(dfj[:, "price"])
filter!(r -> !ismissing(r.bid_buy_size_imbalance_ratio), dfj)
dfj = dfj[(ismissing.(dfj[:, "price"]) |> sum) + 1: end, :]
@assert(ismissing.(dfj[:, "price"]) |> sum == 0)
@assert(ismissing.(dfj[:, "bid_buy_size_imbalance_ratio"]) |> sum == 0)
;

In [ ]:
lt = percentile(dfj[:,"bid_buy_size_imbalance_ratio"], 99.99) # EV distorts chart
gt = percentile(dfj[:,"bid_buy_size_imbalance_ratio"], 0.01) # EV distorts chart
dfp = dfj[gt .< dfj[:,"bid_buy_size_imbalance_ratio"] .< lt , :]
# focus on extreme events
gt = percentile(dfp[:,"bid_buy_size_imbalance_ratio"], 99) # EV distorts chart
lt = percentile(dfp[:,"bid_buy_size_imbalance_ratio"], 1) # EV distorts chart
dfp = dfp[(gt .< dfp[:,"bid_buy_size_imbalance_ratio"]) .| (dfp[:,"bid_buy_size_imbalance_ratio"] .< lt), :]

# Resampling
# dfp[:, "tsg"] .= ceil.(dfp[:, "ts"], Dates.Minute(5))
# dfp = combine(groupby(dfp, "tsg"), "ts" => last, "bid_buy_size_imbalance_ratio" => minimum, "price" => last)[:, ["ts_last", "bid_buy_size_imbalance_ratio_minimum", "price_last"]]
#rename!(dfp, ["ts", "bid_buy_size_imbalance_ratio", "price"]);

print(size(dfp))

In [ ]:
plot([
    scatter(x=dfp[:, "ts"], y=dfp[:, "bid_buy_size_imbalance_ratio"], name="bid_buy_size_imbalance_ratio"),
    scatter(x=dfp[:, "ts"], y=dfp[:, "price"], yaxis="y2", name="price")
], Layout(yaxis2=attr(overlaying="y", side="right")))